In [34]:
from pymongo import MongoClient
import pandas as pd

In [35]:
def download_data_from_mongoatlas(database, table, query={}):
    username = 'entropy_scholar'
    password = 'OCN4kEzbsW5lqQ2W'
    url = 'mongodb+srv://{}:{}@entropy-sfneo.mongodb.net/test?retryWrites=true&w=majority'.format(username, password)
    client = MongoClient(url)
    db = client[database]
    collection = db[table]
    df = pd.DataFrame(list(collection.find(query))).drop('_id', axis=1)
    if 'date' in df.columns:
        try:
            df = df.sort_values(['date','clave_mun'], ascending=True)
        except:
            df = df.sort_values(['date'], ascending=True)
        df = df.reset_index(drop=True)
    return df

In [36]:
intercensal_personas = download_data_from_mongoatlas('intercensal_2015', 'personas')
intercensal_viviendas = download_data_from_mongoatlas('intercensal_2015', 'viviendas')

In [37]:
intercensal_personas_2015 = intercensal_personas[['clave_mun', 'Hombre Escolaridad', 'Mujer Escolaridad']].sort_values('clave_mun')

In [38]:
intercensal_personas_2015

,clave_mun,Hombre Escolaridad,Mujer Escolaridad
0,1001,3126302,3307112
56,1002,114689,127772
3,1003,133447,154546
2,1004,43170,45902
36,1005,373724,387104
...,...,...,...
2445,32054,45553,52418
2432,32055,75851,85033
2447,32056,585820,637862
2452,32057,49714,53118


In [131]:
intercensal_personas_2015['clave_mun'].value_counts()

20478    1
30182    1
24045    1
30188    1
24043    1
        ..
13025    1
15072    1
13023    1
15070    1
20480    1
Name: clave_mun, Length: 2457, dtype: int64

In [39]:
for ind, row in intercensal_personas_2015.iterrows():
    intercensal_personas_2015.loc[ind, "Escolaridad_Total"] = row['Hombre Escolaridad']+ row['Mujer Escolaridad']

In [40]:
intercensal_personas_2015 = intercensal_personas_2015[['clave_mun','Escolaridad_Total']]

In [41]:
intercensal_personas_2015

,clave_mun,Escolaridad_Total
0,1001,6433414.0
56,1002,242461.0
3,1003,287993.0
2,1004,89072.0
36,1005,760828.0
...,...,...
2445,32054,97971.0
2432,32055,160884.0
2447,32056,1223682.0
2452,32057,102832.0


In [42]:
intercensal_viviendas_2015 = intercensal_viviendas[['clave_mun', 'Viviendas con jefe(a) del hogar mujer']].sort_values('clave_mun')

In [43]:
intercensal_viviendas_2015

,clave_mun,Viviendas con jefe(a) del hogar mujer
52,1001,64157
1,1002,2379
31,1003,3630
30,1004,745
43,1005,5751
...,...,...
2196,32054,797
2342,32055,1952
2284,32056,12158
2259,32057,858


In [118]:
municipios = pd.read_csv(r'C:\Users\MICHEL\Documents\pobreza desocupacion.csv')
municipios = municipios.fillna('SIN/DATOS')

In [119]:
municipios

,clave_mun,desocupacion_2015_%,pobreza_2015_%
0,1001,3.9,26.1
1,1002,5.8,54
2,1003,3.7,56.8
3,1004,5.0,43.2
4,1005,2.4,25
...,...,...,...
2452,32054,4.7,71.8
2453,32055,9.3,61
2454,32056,4.2,25.9
2455,32057,5.6,63.1


In [120]:
municipios = municipios.merge(intercensal_viviendas_2015,
            on='clave_mun')

In [121]:
municipios

,clave_mun,desocupacion_2015_%,pobreza_2015_%,Viviendas con jefe(a) del hogar mujer
0,1001,3.9,26.1,64157
1,1002,5.8,54,2379
2,1003,3.7,56.8,3630
3,1004,5.0,43.2,745
4,1005,2.4,25,5751
...,...,...,...,...
2452,32054,4.7,71.8,797
2453,32055,9.3,61,1952
2454,32056,4.2,25.9,12158
2455,32057,5.6,63.1,858


In [48]:
modalidades_agregados = {'homicidio_doloso': ['1HDF', '1HDB', '1HDE', '1HDN'],
                        'extorsion': ['4EEE'],
                        'feminicidio': ['1FFF', '1FFB', '1FFE', '1FFN'],
                        'secuestro': ['2SSE', '2SSC', '2SSD', '2SSX', '2SSO'],
                        'robo_maquinaria': ['4RMA', '4RMB', '4RMC', '4RMD', '4RMF', '4RME'],
                        'robo_negocio': ['4RNC', '4RNS'],
                        'robo_bancos': ['4RBC', '4RBS'],
                        'robo_transportistas': ['4RPC', '4RPS'],
                        'robo_vehiculos': ['4RVA', '4RVB', '4RVM', '4RVN', '4RVE', '4RVF', '4RAC', '4RAS'],
                        'robo_casa': ['4RHC', '4RHS']}

modalidades_nombres = {'homicidio_doloso': 'Homicidio Doloso',
                       'extorsion': 'Extorsión',
                       'feminicidio': 'Feminicidios',
                       'secuestro': 'Secuestro',
                       'robo_maquinaria': 'Robo Maquinaria',
                       'robo_negocio': 'Robo Negocios',
                       'robo_bancos': 'Robo Bancos',
                       'robo_transportistas': 'Robo Transportistas',
                       'robo_vehiculos': 'Robo Vehículos',
                       'robo_casa': 'Robo Casa'}

In [49]:
def extract_df_between_dates(df, start_date, end_date):
    df_fechas = df.loc[(df['date'] >= start_date) & (df['date'] <= end_date)]
    df_fechas = df_fechas.sort_values(['date', 'clave_mun'], ascending=True)
    df_fechas = df_fechas.reset_index(drop=True)    
    return df_fechas

In [50]:
def extract_df_modalidad(df, modalidad):
    df_modalidad = df[['clave_mun'] + modalidades_agregados[modalidad]]
    df_modalidad[modalidad] = [0] * len(df_modalidad)
    for m_code in modalidades_agregados[modalidad]:
        df_modalidad[modalidad] += df_modalidad[m_code]
    df_modalidad = df_modalidad.drop(modalidades_agregados[modalidad], axis=1)
    return df_modalidad

In [51]:
def extract_mpos_poblacion(df, date):
    df_date = df.loc[df['date'] == date]
    df_date = df_date[['clave_mun', 'pob']]
    df_date = df_date.reset_index(drop=True)
    return df_date

In [52]:
snsp_mpos = download_data_from_mongoatlas('sesnsp', 'municipal_carpetas')

In [53]:
start_date = '2015/01'
end_date = '2020/03'
snsp_mpos_dates = extract_df_between_dates(snsp_mpos, start_date, end_date)

In [54]:
snsp_mpos_dates 

,clave_mun,año,clave_ent,mes,pob,date,1HDF,1HDB,1HDE,1HDN,...,7NNN,7AAA,7MMM,7VVV,7FFF,7III,7CCC,7PPP,7EEE,7OOO
0,1001,2015,01,1,877190.0,2015/01,2.0,1.0,0.0,1.0,...,26.0,87.0,12.0,0.0,4.0,27.0,0.0,1.0,0.0,63.0
1,1002,2015,01,1,46464.0,2015/01,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1003,2015,01,1,56048.0,2015/01,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
3,1004,2015,01,1,15577.0,2015/01,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1005,2015,01,1,120405.0,2015/01,1.0,0.0,0.0,0.0,...,3.0,5.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142648,32054,2020,32,3,19155.0,2020/03,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
142649,32055,2020,32,3,30240.0,2020/03,0.0,0.0,1.0,0.0,...,1.0,4.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0
142650,32056,2020,32,3,146147.0,2020/03,0.0,0.0,0.0,0.0,...,4.0,23.0,4.0,0.0,3.0,1.0,0.0,10.0,0.0,30.0
142651,32057,2020,32,3,19413.0,2020/03,0.0,0.0,0.0,0.0,...,1.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [55]:
snsp_mpos_dates = snsp_mpos_dates[['clave_mun', 'date','1FFF', '1FFB', '1FFE', '1FFN']]

In [69]:
snsp_mpos_dates

,clave_mun,date,1FFF,1FFB,1FFE,1FFN
0,1001,2015/01,0.0,0.0,0.0,0.0
1,1002,2015/01,0.0,0.0,0.0,0.0
2,1003,2015/01,0.0,0.0,0.0,0.0
3,1004,2015/01,0.0,0.0,0.0,0.0
4,1005,2015/01,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
142648,32054,2020/03,0.0,0.0,0.0,0.0
142649,32055,2020/03,0.0,0.0,0.0,0.0
142650,32056,2020/03,0.0,0.0,0.0,0.0
142651,32057,2020/03,0.0,0.0,0.0,0.0


In [70]:
snsp_mpos_agg = snsp_mpos_dates.groupby(['clave_mun']).sum()
snsp_mpos_agg = snsp_mpos_agg.reset_index()

In [73]:
snsp_mpos_agg_feminicidio = extract_df_modalidad(snsp_mpos_agg, 'feminicidio')

In [74]:
snsp_mpos_agg_feminicidio

,clave_mun,feminicidio
0,1001,5.0
1,1002,1.0
2,1003,0.0
3,1004,0.0
4,1005,3.0
...,...,...
2466,32054,0.0
2467,32055,1.0
2468,32056,4.0
2469,32057,0.0


In [75]:
mpos_poblacion = extract_mpos_poblacion(snsp_mpos, '2020/03')

In [76]:
mpos_poblacion

,clave_mun,pob
0,1001,877190.0
1,1002,46464.0
2,1003,56048.0
3,1004,15577.0
4,1005,120405.0
...,...,...
2466,32054,19155.0
2467,32055,30240.0
2468,32056,146147.0
2469,32057,19413.0


In [77]:
snsp_mpos_agg_feminicidio = snsp_mpos_agg_feminicidio.merge(mpos_poblacion,
                                                          on='clave_mun')

In [78]:
def fem_tasa_100mil(df, column):
    tasas = []
    for i in range(len(df)):
        f = df[column][i]
        p = df['pob'][i]
        f_tasa = f / (p / 100000)
        tasas.append(f_tasa)
    df[column + '_tasa'] = tasas
    return df

In [79]:
snsp_mpos_agg_feminicidio = fem_tasa_100mil(snsp_mpos_agg_feminicidio, 'feminicidio')

In [80]:
snsp_mpos_agg_feminicidio

,clave_mun,feminicidio,pob,feminicidio_tasa
0,1001,5.0,877190.0,0.570002
1,1002,1.0,46464.0,2.152204
2,1003,0.0,56048.0,0.000000
3,1004,0.0,15577.0,0.000000
4,1005,3.0,120405.0,2.491591
...,...,...,...,...
2466,32054,0.0,19155.0,0.000000
2467,32055,1.0,30240.0,3.306878
2468,32056,4.0,146147.0,2.736970
2469,32057,0.0,19413.0,0.000000


In [100]:
Y = pd.read_csv(r'C:\Users\MICHEL\Documents\y feminicidios.csv')

In [110]:
Y = Y[['clave_mun', 'Y']]

In [111]:
Y = Y.merge(intercensal_personas_2015,
            on='clave_mun')

In [112]:
Y_final = Y.merge(municipios,
           on='clave_mun')

In [113]:
Y_final

,clave_mun,Y,Escolaridad_Total,desocupacion_2015_%,pobreza_2015_%,Viviendas con jefe(a) del hogar mujer
0,1001,0,6433414.0,3.9,26.1,64157
1,1002,1,242461.0,5.8,54,2379
2,1003,0,287993.0,3.7,56.8,3630
3,1004,0,89072.0,5.0,43.2,745
4,1005,1,760828.0,2.4,25,5751
...,...,...,...,...,...,...
2452,32054,0,97971.0,4.7,71.8,797
2453,32055,1,160884.0,9.3,61,1952
2454,32056,1,1223682.0,4.2,25.9,12158
2455,32057,0,102832.0,5.6,63.1,858


In [130]:
Y_final['clave_mun'].value_counts()

20478    1
30182    1
24045    1
30188    1
24043    1
        ..
13025    1
15072    1
13023    1
15070    1
20480    1
Name: clave_mun, Length: 2457, dtype: int64

In [86]:
Y_final.to_excel('Feminicidios_base.xlsx')

# ESTADÍSTICAS DESCRIPTIVAS

In [122]:
Y_final.columns

Index(['clave_mun', 'Y', 'Escolaridad_Total', 'desocupacion_2015_%',
       'pobreza_2015_%', 'Viviendas con jefe(a) del hogar mujer'],
      dtype='object')

In [123]:
Y_final['Viviendas con jefe(a) del hogar mujer'].describe()

count      2457.000000
mean       3771.351648
std       11996.462741
min          10.000000
25%         275.000000
50%         845.000000
75%        2306.000000
max      175491.000000
Name: Viviendas con jefe(a) del hogar mujer, dtype: float64

In [127]:
Y_final['Escolaridad_Total'].describe()

count    2.457000e+03
mean     3.211430e+05
std      1.065962e+06
min      0.000000e+00
25%      1.893900e+04
50%      6.422600e+04
75%      1.758690e+05
max      1.439412e+07
Name: Escolaridad_Total, dtype: float64

In [128]:
Y_final['desocupacion_2015_%'].describe()

count    2457.000000
mean        4.550387
std         3.983558
min         0.000000
25%         2.400000
50%         3.700000
75%         5.500000
max        51.900000
Name: desocupacion_2015_%, dtype: float64

In [124]:
Y_final['pobreza_2015_%'].describe()

count     2457
unique     777
top       60.2
freq        12
Name: pobreza_2015_%, dtype: object

## RESUMEN DE ESTADÍSTICAS DESCRIPTIVAS

In [125]:
Y_final.describe()

,clave_mun,Y,Escolaridad_Total,desocupacion_2015_%,Viviendas con jefe(a) del hogar mujer
count,2457.000000,2457.000000,2.457000e+03,2457.000000,2457.000000
mean,19368.536427,0.339031,3.211430e+05,4.550387,3771.351648
std,7360.585604,0.473477,1.065962e+06,3.983558,11996.462741
min,1001.000000,0.000000,0.000000e+00,0.000000,10.000000
25%,14084.000000,0.000000,1.893900e+04,2.400000,275.000000
50%,20231.000000,0.000000,6.422600e+04,3.700000,845.000000
75%,24030.000000,1.000000,1.758690e+05,5.500000,2306.000000
max,32058.000000,1.000000,1.439412e+07,51.900000,175491.000000
